In [ ]:
#import libraries
import requests 
import pandas as pd
from bs4 import BeautifulSoup 
import csv 
import lxml
from textblob import TextBlob

In [ ]:
#get html
URL = "https://www.isaaa.org/gmapprovaldatabase/geneslist/default.asp"
gmo = requests.get(URL) 
print(gmo.content) 

In [ ]:
soup = BeautifulSoup(gmo.content, "lxml")
print(soup.prettify()) 

In [ ]:
xml_soup = BeautifulSoup(gmo.content, 'xml')
print(xml_soup.prettify())

In [ ]:
soup.p.get_attribute_list('contenttext')

In [ ]:
soup.title.string

In [ ]:
table.find_all('td')

In [ ]:
souptext = str(soup.get_text())
blobtext = TextBlob(souptext)
print(blobtext.parse())

In [ ]:
smalltext_soup = soup.find_all("tr", class_="smalltext")
for textdata in smalltext_soup: 
    text = str(textdata.text)
    genetext_raw = souptext.splitlines() 
    print(genetext_raw)

## Scrapwork

In [ ]:
str_gt = str(genetext_raw)
df = pd.DataFrame([x.split(',') for x in str_gt.split('\n')])
pd.DataFrame.dropna(df, how = 'all')

In [ ]:
gene_names = []
gmo_tds = soup("tr", class_="smalltext")
for td in gmo_tds:
    gene_names.append(td.find("a").contents[0])
gene_names

In [ ]:
souptext = print(soup.get_text())

In [ ]:
soup = BeautifulSoup('<td bgcolor="#F7F7F7" valign="top"><strong>Product:</strong> CAld5H enzyme <br/>')
tag = soup.b
type(tag)

In [ ]:
souptext.splitlines()

In [ ]:
splitli

In [ ]:
str(souptext)

In [ ]:
for gene in 

In [ ]:
classes = []
for element in soup.find_all(class_=True):
    classes.extend(element["class"])
print(classes)